In [ ]:
!pip install -q gradio requests

In [ ]:
!pip show gradio
!pip show requests

Name: gradio
Version: 5.50.0
Summary: Python library for easily interacting with trained machine learning models
Home-page: https://github.com/gradio-app/gradio
Author: 
Author-email: Abubakar Abid <gradio-team@huggingface.co>, Ali Abid <gradio-team@huggingface.co>, Ali Abdalla <gradio-team@huggingface.co>, Dawood Khan <gradio-team@huggingface.co>, Ahsen Khaliq <gradio-team@huggingface.co>, Pete Allen <gradio-team@huggingface.co>, Ömer Faruk Özdemir <gradio-team@huggingface.co>, Freddy A Boulton <gradio-team@huggingface.co>, Hannah Blair <gradio-team@huggingface.co>
License: 
Location: /usr/local/lib/python3.12/dist-packages
Requires: aiofiles, anyio, brotli, fastapi, ffmpy, gradio-client, groovy, httpx, huggingface-hub, jinja2, markupsafe, numpy, orjson, packaging, pandas, pillow, pydantic, pydub, python-multipart, pyyaml, ruff, safehttpx, semantic-version, starlette, tomlkit, typer, typing-extensions, uvicorn
Required-by: 
Name: requests
Version: 2.32.4
Summary: Python HTTP for Human

In [ ]:
import gradio as gr
import requests
import json
import socket

In [ ]:
NGROK_PUBLIC_URL = "https://autoicous-daine-homelike.ngrok-free.dev"

def build_base_url(host:str,port:str):
  return host.rstrip("/")

def can_connect(host:str,port:str,timeout:float=1.0):
  try:
    response = requests.get(f"{host.rstrip('/')}/",timeout=5)
    return response.status_code == 200
  except Exception:
    return False

def call_query_index(host:str,port:str,query:str):
  url = build_base_url(host,port) + "/query_index"
  print(f"url: {url}")

  #post on url
  try:
    resp = requests.post(url,json={"query":query},timeout=60)
  except requests.exceptions.RequestException as e:
    err = f"**Request Failed!** cannot connect to {url}. \
    Please ensure your RAG script is running and its Ngrok tunnel is active. Error: {e}"
    return err, json.dumps({"error": str(e),"url":url},indent=2)
  #json
  try:
    data = resp.json()
    """
    .json() is python object
    dumps() converts to json
    loads() converts to python
    """
  except Exception:
    return f"**Invalid JSON Response** (Status: {resp.status_code}):\n{resp.text}",\
    json.dumps({"status_code":resp.status_code,"text":resp.text})
  #get the data
  if resp.status_code == 200:
    pretty_answer = data.get('index_response','No response found in index_response field')
    # index_response is the key in the dictionary
    final_output = f"**RAG Agent Response**\n\n{pretty_answer}"
  else:
    final_output = f"**Server Error** (Status: {resp.status_code}): {data.get('detail',data)}"
    # FastAPI automatically uses "detail" as the key when returning errors.
  raw_json = json.dumps(data,indent=2)
  return final_output,raw_json


def call_get_branch_info(host:str,port:str,zip_code:str):
  url = build_base_url(host,port) + "/get_branch_info"

  try:
    resp = requests.get(url,params={"zip_code":zip_code},timeout=20)
  except requests.exceptions.RequestException as e:
    err = f"**Request Failed!** Cannot connect to {url}. Please ensure your RAG script is running and its Ngrok tunnel is active. Error: {e}"
    return err, json.dumps({'error':str(e),'url':url},indent=2)

  try:
    data = resp.json()
  except Exception:
    return f"**Invalid JSON Response** (Status {resp.status_code}):\n{resp.text}", json.dumps({"status_code": resp.status_code, "text": resp.text}, indent=2)

  if resp.status_code == 200:
    pretty_address = data.get('branch_address','Branch address field not found in response.')
  else:
    pretty_address = f"**Server Error** (Status {resp.status_code}): {data.get('detail',data)} "

  raw_json = json.dumps(data,indent=2)
  return pretty_address,raw_json


# --- GRADIO HANDLERS ---
def handle_query(query:str,host:str,port:str):
  if not query or query.strip() == "":
    return "Please type a question.",gr.update(visible=False,value={})
    # return - first is the markdown, second is json

  if not can_connect(host,port): #boolean is returned-if False,(not False = True) then below return is run
    return f"**Connection Error:** Cannot reach the Ngrok URL {host}. \
    Please verify the URL and ensure your RAG script is still running!", \
    gr.update(visible=False, value={})

  pretty_output, raw_output = call_query_index(host,port,query)

  return pretty_output,gr.update(value=json.loads(raw_output))
  """
  value=json.loads(raw_output) - convert to python dict
  raw_output -- The raw output you receive is a JSON string
  gr.JSON expects a Python dictionary, not a string
  """

def handle_branch(zip_code:str,host:str,port:str):
  if not zip_code:
    return "Please provide zip code", gr.update(visible=False, value={})

  if not can_connect(host,port):
    return f"**Connection Error:** Cannot reach the Ngrok URL {host}. \
    Please verify the URL and ensure your RAG script is still running!", \
    gr.update(visible=False, value={})

  pretty_output, raw_output = call_get_branch_info(host,port,zip_code)

  return pretty_output,gr.update(value=json.loads(raw_output))

# --- GRADIO UI LAYOUT ---

with gr.Blocks(title='URA Bank - RAG Agent UI Client') as demo:
  gr.Markdown(f"""
  URA Bank - RAG AI Agent Client (Gradio)
  This client connects to your FastAPI server via the **Ngrok Public URL** for maximum reliability in Colab.

  **Instructions:**
  1. **Start your RAG script** and ensure it prints the Ngrok URL.
  2. **Ensure the Host below is your current Ngrok URL.**
  """)

  with gr.Row():
    api_host = gr.Textbox(label="FastAPI Host (Full Ngrok HTTPS URL)",value=NGROK_PUBLIC_URL,scale=3)
    #url is passed here
    api_port = gr.Textbox(label="FastAPI Port (Ignored for Ngrok)",value="443",scale=1,interactive=False)
    show_raw_checkbox = gr.Checkbox(label="Show Raw JSON Response",value=False,scale=0)

  with gr.Tabs():

    with gr.TabItem("RAG Knowledge Query (/query_index)"):
      rag_query_input = gr.Textbox(
          label="RAG Query",
          placeholder="e.g., What are the terms for the Unsecured Personal Loan?",
          lines=3
          )
      rag_button = gr.Button("Get RAG Answer",variant="primary")
      # variant → controls the visual style of the button.
      # "primary"   → main action button (highlighted / most important).
      # "secondary" → less important button (lighter style).
      rag_output = gr.Markdown(label='RAG Agent Answer')
      # rag_output = gr.Markdown(label="RAG Agent Answer", value="")  # forces box to appear
      # label ≠ visible content
      rag_raw_output = gr.JSON(label="Raw JSON Response",visible=False,value={})

      rag_button.click(
          fn=handle_query,
          inputs=[rag_query_input,api_host,api_port],
          outputs=[rag_output,rag_raw_output]
          )

    with gr.TabItem("Branch Locator Tool (/get_branch_info)"):
      zip_code_input = gr.Textbox(
          label="Enter Zip Code",
          placeholder="e.g.,500034",
          lines=1
          )
      branch_button = gr.Button("Find Nearest Branch",variant="secondary")
      branch_output = gr.Markdown(label="Nearest Branch Address")
      branch_raw_output = gr.JSON(label="Raw JSON Response",visible=False,value={})

      branch_button.click(
          fn=handle_branch,
          inputs=[zip_code_input,api_host,api_port],
          outputs=[branch_output,branch_raw_output]
      )

    def update_raw_visibility(show): # boolean
      return (
          gr.update(visible=show), # for rag_raw_output
          gr.update(visible=show) # for branch_raw_output
      )

    show_raw_checkbox.change(
        fn=update_raw_visibility,
        inputs=[show_raw_checkbox],  # checkbox input
        outputs=[rag_raw_output,branch_raw_output] #json output for rag_query and branch address
    )

if __name__ == '__main__':
  demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://eb5d3cf73e569664cf.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
